BASIC GEN AI APP USING LANGCHAIN

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [ ]:
#Data ingestion - scrapping web page
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.langchain.com/langsmith/observability-quickstart")

In [ ]:
docs = loader.load()
docs[0]

In [ ]:
#split the data into chunks using recursive character text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)
(texts)

In [ ]:
# embedding and vector store
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)
db

In [ ]:
#similarity search
query = "how langchain provides end-to-end visibility "
result=db.similarity_search(query)
result[0].page_content

In [ ]:
#openai LLM
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")


In [ ]:
## Retrieval Chain, Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following question using only the provided context.
If the context does not contain the answer, say "I don’t know."

<context>
{context}
</context>

Question: {input}"""
)
doc_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
doc_chain

In [ ]:
# Here we are manually passing both:
# 1. The user question ("input")
# 2. The supporting context ("context") as a list of Document objects
# This simulates what a retriever would normally do automatically.
# The doc_chain will stuff the docs into the prompt alongside the question
# and then send everything to the LLM.
from langchain_core.documents import Document
doc_chain.invoke({
    "input":query,
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

In [ ]:
#Retriever to get context from vector store and pass to llm
retriever=db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,doc_chain)


In [ ]:
retrieval_chain

In [ ]:
#output from llm
result=retrieval_chain.invoke({"input": "are llms are non-deterministic"})
result['answer']